# Teoría Elemental de Números con SageMath

Este cuaderno cubre nociones básicas y prácticas:
- Algoritmo de división
- Máximo común divisor (MCD) y algoritmo de Euclides
- Algoritmo de Euclides extendido y coeficientes de Bézout
- Inverso modular y congruencias
- Teorema Chino del Resto (TCR)
- Potenciación modular
- (Bonus) MCD de polinomios

**Requisitos**: Seleccioná el *kernel* **SageMath** si Jupyter te lo pide.


In [1]:
# Verificar versión de Sage y entorno
version(), sys.version


('SageMath version 10.4, Release Date: 2024-07-19',
 '3.12.4 (main, Jul 20 2024, 01:19:25) [GCC 11.4.0]')

## 1) Algoritmo de división

Para enteros \(a\) y \(b\neq 0\), existen únicos \(q, r\) tales que
\[ a = b\,q + r,\quad 0 \le r < |b|. \]
En Sage, la división entera usa `//` y el resto `%`. Vamos a empaquetarlo y validar las condiciones.

In [2]:
def division_algorithm(a, b):
    if b == 0:
        raise ZeroDivisionError("b no puede ser 0")
    q = a // b
    r = a % b
    # Asegurar la convención 0 <= r < |b|
    # En Sage/% Python, para enteros, ya se cumple esta convención.
    return q, r

tests = [(23, 5), (23, -5), (-23, 5), (-23, -5), (100, 7)]
[(a, b, ) + division_algorithm(a, b) for a, b in tests]

[(23, 5, 4, 3),
 (23, -5, -5, -2),
 (-23, 5, -5, 2),
 (-23, -5, 4, -3),
 (100, 7, 14, 2)]

In [3]:
# Validación automática de la propiedad a = b*q + r, 0 <= r < |b|
for a, b in tests:
    q, r = division_algorithm(a, b)
    assert a == b*q + r, (a,b,q,r)
    assert 0 <= r < abs(b), (a,b,q,r)
print("OK división: todas las pruebas pasaron.")

AssertionError: (23, -5, -5, -2)

## 2) MCD y algoritmo de Euclides
El **MCD** puede calcularse con el algoritmo de Euclides, usando divisiones sucesivas.
Sage provee `gcd(a,b)`, pero lo implementamos para ver los pasos.

In [4]:
def euclid_gcd(a, b, trace=True):
    a, b = ZZ(a), ZZ(b)
    if trace: print(f"Euclides para gcd({a}, {b})")
    while b != 0:
        q, r = a // b, a % b
        if trace: print(f"{a} = {b}*{q} + {r}")
        a, b = b, r
    if trace: print(f"gcd = {a}")
    return int(a)

g1 = euclid_gcd(544, 119)
g2 = gcd(544, 119)
g1, g2

Euclides para gcd(544, 119)
544 = 119*4 + 68
119 = 68*1 + 51
68 = 51*1 + 17
51 = 17*3 + 0
gcd = 17


(17, 17)

## 3) Euclides extendido y coeficientes de Bézout
Queremos \(x, y\) tales que \(ax + by = \gcd(a,b)\). Sage da esto con `xgcd(a,b)`.

In [6]:
def extended_euclid(a, b):
    # Implementación manual
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r != 0:
        q = old_r // r
        old_r, r = r, old_r - q*r
        old_s, s = s, old_s - q*s
        old_t, t = t, old_t - q*t
    # old_r = gcd(a,b), old_s = x, old_t = y
    return old_r, old_s, old_t

a, b = 400, 14
g, x, y = extended_euclid(a, b)
gx, ux, vx = xgcd(a, b)  # Sage: (g, u, v)
print("Manual:", g, x, y, "→", a*x + b*y)
print("Sage  :", gx, ux, vx, "→", a*ux + b*vx)

Manual: 2 2 -57 → 2
Sage  : 2 2 -57 → 2


## 4) Inverso modular y congruencias
Para \(a\) invertible módulo \(m\) (i.e. \(\gcd(a,m)=1\)), el inverso se obtiene con Euclides extendido.
En Sage: `inverse_mod(a, m)`.

In [7]:
def inv_mod_via_egcd(a, m):
    g, x, y = extended_euclid(a, m)
    if g != 1:
        raise ValueError(f"No existe inverso: gcd({a},{m})={g}")
    return x % m

a, m = 17, 43
inv1 = inv_mod_via_egcd(a, m)
inv2 = inverse_mod(a, m)
print(inv1, inv2, (a*inv1) % m)

# Resolver a*x ≡ b (mod m): x ≡ b * a^{-1} mod m
b = 7
x_sol = (b * inverse_mod(a, m)) % m
x_sol, (a*x_sol - b) % m

38 38 1


(8, 0)

## 5) Teorema Chino del Resto (TCR)
Si \(m, n\) son coprimos, existe solución única modulo \(mn\) a:
\[ x \equiv a \;(\text{mod } m),\quad x \equiv b \;(\text{mod } n). \]
En Sage: `crt(a, b, m, n)`.

In [ ]:
m, n = 14, 25
a, b = 5, 17
x = crt(a, b, m, n)
x, x % m, x % n, lcm(m, n)

## 6) Potenciación modular (rápida)
Sage trae `power_mod(base, exp, mod)`.

In [ ]:
power_mod(7, 560, 561), pow(7, 560, 561)  # Carmichael 561: pseudo-primo de Fermat para base 2


## 7) (Bonus) MCD de polinomios
En \(\mathbb{Q}[x]\) o \(\mathbb{Z}[x]\) el algoritmo de Euclides funciona igual. Usemos `gcd` y `xgcd`.

In [ ]:
R.<x> = QQ[]
P = (x^4 - 1)
Q = (x^3 - 1)
gP = gcd(P, Q)
gP, factor(gP)

# Coeficientes de Bézout en polinomios
g, U, V = xgcd(P, Q)
g, (U*P + V*Q).simplify_full()


## 8) Ejercicios propuestos (con tests)
**A.** Implementá `my_gcd(a, b)` usando restas en lugar de divisiones.

**B.** Escribí `solve_linear_congruence(a, b, m)` que devuelva la lista de soluciones \(x\) de \(a x \equiv b\ (\bmod\ m)\). Pista: usá `g = gcd(a,m)`.

**C.** Verificá computacionalmente que `gcd(F_m, F_n) == F_gcd(m, n)` para \(m, n\) al azar (Fibonacci).

In [ ]:
def my_gcd(a, b):
    a, b = abs(a), abs(b)
    if a == 0: return b
    if b == 0: return a
    while a != b:
        if a > b:
            a = a - b
        else:
            b = b - a
    return a

assert my_gcd(544, 119) == gcd(544,119)
assert my_gcd(0, 10) == 10 and my_gcd(10, 0) == 10
print("OK my_gcd")

In [ ]:
def solve_linear_congruence(a, b, m):
    a, b, m = ZZ(a), ZZ(b), ZZ(m)
    g = gcd(a, m)
    if b % g != 0:
        return []  # sin soluciones
    # Reducir
    a1, b1, m1 = a//g, b//g, m//g
    x0 = (b1 * inverse_mod(a1, m1)) % m1
    # g soluciones: x = x0 + k*(m1), k=0..g-1
    return [(x0 + k*m1) % m for k in range(g)]

sols = solve_linear_congruence(14, 30, 100)
sols, [ (14*x - 30) % 100 for x in sols ]

In [ ]:
def F(n):
    # Fibonacci simple
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a+b
    return a

import random
for _ in range(10):
    m = randint(1, 200)
    n = randint(1, 200)
    assert gcd(F(m), F(n)) == F(gcd(m, n))
print("OK identidad gcd(F_m, F_n) = F_{gcd(m,n)}")

## 9) Mini caso práctico: clave modular (estilo RSA de juguete)
1. Elegimos primos pequeños \(p, q\), \(n = pq\).
2. \(\varphi(n) = (p-1)(q-1)\).
3. Elegimos \(e\) coprimo con \(\varphi(n)\) y calculamos \(d = e^{-1} \bmod \varphi(n)\).
4. Ciframos \(c \equiv m^e \bmod n\) y desciframos \(m \equiv c^d \bmod n\).

In [ ]:
p, q = 101, 113
n = p*q
phi = (p-1)*(q-1)
e = 17
d = inverse_mod(e, phi)
m = 1234
c = power_mod(m, e, n)
m_rec = power_mod(c, d, n)
n, phi, e, d, c, m_rec

---
### Sugerencias
- Probá `gcd` y `xgcd` con números grandes.
- Experimentá con `GF(p)` (campos finitos) para aritmética modular más estructurada.
- Para polinomios sobre campos finitos: `GF(101)['x']`.
